## **Importin Libraries & Datas**

In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.image import resize
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
classes = ['rock', 'reggae', 'pop', 'metal', 'jazz', 'hiphop', 'disco', 'country', 'classical', 'blues']

In [3]:
from tensorflow.keras.models import load_model
loadedModel = load_model('/content/drive/My Drive/Music Genre Classification/Models/Trained_model.keras')

## **Single Audio Preprocess**

In [4]:
def loadAndPreprocessData(filePath, targetShape = (150, 150)):
  data = []
  audioData, sampleRate = librosa.load(filePath, sr = None)
  chunkDuration = 4
  overlapDuration = 2
  chunkSamples = chunkDuration * sampleRate
  overlapSamples = overlapDuration * sampleRate
  numChunks = int(np.ceil((len(audioData) - chunkSamples) / (chunkSamples - overlapSamples))) + 1

  for i in range(numChunks):
    start = i * (chunkSamples - overlapSamples)
    end = start + chunkSamples
    chunk = audioData[start:end]

    start_time_sec = start / sampleRate
    end_time_sec = end / sampleRate

    spectrogram = librosa.feature.melspectrogram(y=chunk, sr = sampleRate)

    resizedSpectrogram = resize(np.expand_dims(spectrogram, axis = -1), targetShape)

    data.append(resizedSpectrogram)

  return np.array(data)

In [8]:
filePath = "/content/drive/MyDrive/Music Genre Classification/Data/Test Data/Amour interdit (forbidden love) - Eric Christian.wav"

In [ ]:
from IPython.display import Audio
y, sr = librosa.load(filePath, sr=44100)
Audio(data=y, rate=sr)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
XTest = loadAndPreprocessData(filePath)

In [10]:
XTest.shape

(62, 150, 150, 1)

## **Model Prediction**

In [11]:
def ModelPrediction(XTest):
  yPred = loadedModel.predict(XTest)
  predictedCategoris = np.argmax(yPred, axis = 1)
  uniqueElements, counts = np.unique(predictedCategoris, return_counts = True)
  maxCount = np.max(counts)
  maxElements = uniqueElements[counts == maxCount]
  return maxElements[0]

In [12]:
categoryIndex = ModelPrediction(XTest)

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step


In [13]:
classes[categoryIndex]

'country'

In [14]:
fileDirectory = "/content/drive/MyDrive/Music Genre Classification/Data/Test Data/"
wavFiles = [f for f in os.listdir(fileDirectory) if f.endswith('.wav')]

for file in wavFiles:
    filePath = os.path.join(fileDirectory, file)
    XTest = loadAndPreprocessData(filePath)
    categoryIndex = ModelPrediction(XTest)
    print(f"{file} : {classes[categoryIndex]}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
La Solitude.wav : classical
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 467ms/step
Damn Your Eyes.wav : rock
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Saun & Starr In The Night.wav : pop
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Bob Marley & The Wailers - Buffalo Soldier (Official Music Video).wav : reggae
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 890ms/step
Ajda Pekkan - Yaz Yaz Yaz.wav : reggae
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 465ms/step
Mein Herz brennt.wav : rock
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
And I Love Her.wav : classical
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 663ms/step
Nazan Öncel - Beni Hatırla.wav : pop
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Louis Armstrong - La vie en rose.wav : jazz
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
My Morning Jacket - Dondante.wav : country
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Chet Baker - I fall in love too easily.wav : jazz
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 989ms/step
I'd Rather Go Blind.wav : blues
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Amour interdit (forbidden love) -

In [15]:
import os
import numpy as np

def ModelPredictionPercentage(XTest):
    yPred = loadedModel.predict(XTest, verbose=0)
    meanPred = np.mean(yPred, axis=0)
    top3_indices = np.argsort(meanPred)[-3:][::-1]

    top3_probs = meanPred[top3_indices] * 100
    return list(zip(top3_indices, top3_probs))

fileDirectory = "/content/drive/MyDrive/Music Genre Classification/Data/Test Data/"
wavFiles = [f for f in os.listdir(fileDirectory) if f.endswith('.wav')]

for file in wavFiles:
    filePath = os.path.join(fileDirectory, file)
    XTest = loadAndPreprocessData(filePath)
    predictions = ModelPredictionPercentage(XTest)
    output = f"{file} : "
    for idx, prob in predictions:
        output += f"{classes[idx]} ({prob:.2f}%), "
    print(output[:-2])  # Son virgülü kaldır
    print(" ")


La Solitude.wav : classical (74.49%), rock (11.41%), jazz (8.58%)
 
Damn Your Eyes.wav : rock (58.23%), country (23.21%), pop (10.63%)
 
Saun & Starr In The Night.wav : pop (28.42%), blues (18.24%), reggae (17.64%)
 
Bob Marley & The Wailers - Buffalo Soldier (Official Music Video).wav : reggae (90.65%), disco (3.16%), rock (1.83%)
 
Ajda Pekkan - Yaz Yaz Yaz.wav : reggae (66.40%), rock (12.12%), metal (6.72%)
 
Mein Herz brennt.wav : rock (32.94%), metal (22.49%), pop (18.41%)
 
And I Love Her.wav : classical (42.20%), rock (13.90%), jazz (13.55%)
 
Nazan Öncel - Beni Hatırla.wav : pop (72.29%), rock (19.17%), hiphop (2.36%)
 
Louis Armstrong - La vie en rose.wav : jazz (34.74%), classical (22.77%), blues (16.51%)
 
My Morning Jacket - Dondante.wav : rock (29.91%), country (26.70%), jazz (13.36%)
 
Chet Baker - I fall in love too easily.wav : country (33.13%), jazz (31.54%), blues (25.15%)
 
I'd Rather Go Blind.wav : blues (27.46%), country (26.91%), pop (17.76%)
 
Amour interdit (fo